### A. Starting Kafka and Zookeeper

If you haven't already started Kafka and Zookeeper, you need to do so:

1. **Start Zookeeper**:
   ```sh
   bin/zookeeper-server-start.sh config/zookeeper.properties
   ```

2. **Start Kafka**:
   ```sh
   bin/kafka-server-start.sh config/server.properties
   ```
3. **Create a Topic**:
   ```sh
   bin/kafka-topics.sh --create --topic input-events --bootstrap-server localhost:9092
   ```

Great! You've successfully created a Kafka topic named `quick-events`. The next steps typically involve producing some messages to this topic and then consuming those messages. Here are the commands for both:

### Producing Messages to the Topic

1. **Start a Kafka Producer**:
   ```sh
   bin/kafka-console-producer.sh --topic quick-events --bootstrap-server localhost:9092
   ```
   This command will open a prompt where you can start typing messages. Each line you type will be sent as a message to the `quick-events` topic. To exit, press `Ctrl+C`.

### Consuming Messages from the Topic

2. **Start a Kafka Consumer**:
   ```sh
   bin/kafka-console-consumer.sh --topic quick-events --from-beginning --bootstrap-server localhost:9092
   ```
   This command will start a consumer that reads messages from the `quick-events` topic starting from the beginning.


  

These commands should be run in separate terminal windows to keep Zookeeper and Kafka running.

### Summary of Commands


1. **Start Zookeeper (if not already running)**:
   ```sh
   bin/zookeeper-server-start.sh config/zookeeper.properties
   ```

2. **Start Kafka (if not already running)**:
   ```sh
   bin/kafka-server-start.sh config/server.properties
   ```

3. **Create a Topic**:
   ```sh
   bin/kafka-topics.sh --create --topic quick-events --bootstrap-server localhost:9092
   ```

4. **Start a Producer**:
   ```sh
   bin/kafka-console-producer.sh --topic quick-events --bootstrap-server localhost:9092
   ```

5. **Start a Consumer**:
   ```sh
   bin/kafka-console-consumer.sh --topic quick-events --from-beginning --bootstrap-server localhost:9092
   ```


To set the `PYSPARK_SUBMIT_ARGS` environment variable with the appropriate Kafka package for Spark 3.5.1 and Scala 2.12.18, and Kafka 2.13-3.7.1, you need to ensure that the Kafka connector matches your Spark and Scala versions. Since your Spark version is 3.5.1 and it uses Scala 2.12, you need to use the Kafka connector for Scala 2.12. 

Here’s how to set it up:

### B. Setting Up Your Jupyter Notebook


1. **Create a Spark Session**:
   ```python
   from pyspark.sql import SparkSession

   spark = SparkSession.builder \
       .appName("KafkaExample") \
       .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")
       .getOrCreate()
   ```

2. **Reading from Kafka**:
   ```python
   kafka_df = spark.readStream \
       .format("kafka") \
       .option("kafka.bootstrap.servers", "localhost:9092") \
       .option("subscribe", "quick-events") \
       .load()

   kafka_df = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
   ```

3. **Writing to Kafka**:
   ```python
   data = [("key1", "message1"), ("key2", "message2")]
   columns = ["key", "value"]
   df = spark.createDataFrame(data, columns)
   kafka_df_write = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

   kafka_df_write.write \
       .format("kafka") \
       .option("kafka.bootstrap.servers", "localhost:9092") \
       .option("topic", "quick-events") \
       .save()
   ```

### Full Example Notebook

Here’s the complete example in a Jupyter Notebook cell, ensuring compatibility:

```python

# Import SparkSession
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaExample") \
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")
    .getOrCreate()

# Verify Spark version
print(f"Spark version: {spark.version}")

# Reading from Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "quick-events") \
    .load()

kafka_df = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Write to Console (streaming)
query = kafka_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Example data to write to Kafka
data = [("key1", "message1"), ("key2", "message2")]
columns = ["key", "value"]
df = spark.createDataFrame(data, columns)
kafka_df_write = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Write to Kafka
kafka_df_write.write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "quick-events") \
    .save()

query.awaitTermination()
```

### Notes

- **Ensure Kafka Server is Running**: Make sure your Kafka server is up and running at `localhost:9092`.
- **Version Compatibility**: Double-check that the versions of Spark, Scala, and the Kafka connector are compatible. The key here is to match the Scala version used by Spark.
- **Network Access**: Ensure your environment has network access to download dependencies when using the `--packages` option.

By including the correct Kafka package and Scala library for your Spark and Scala versions, the missing class definition error should be resolved. If you continue to encounter issues, please provide further details for additional troubleshooting.